In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.amp import autocast, GradScaler
from torch.utils.data import DataLoader, TensorDataset
import optuna
import netron
import xarray as xr

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA Device: {torch.cuda.get_device_name(device)}")
else:
    print("CUDA is not available")



ModuleNotFoundError: No module named 'optuna'

In [1]:
from datetime import datetime
# Function to log messages with timestamps
def log(msg):
    print(f"[{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] {msg}")

In [3]:
import numpy as np
import xarray as xr

log("Starting data loading...")

# Paths to your NetCDF files
paths = "/run/media/adamh/Elements/training_data/era5_model_levels_*.nc"

# Open and combine all files
log("Opening and combining NetCDF files...")
ds = xr.open_mfdataset(paths, combine='by_coords')

# Extract temperature
log("Extracting temperature variable...")
temp = ds['t']  # adjust if needed

# Select model levels 58 to 137
log("Selecting model levels 58 to 137...")
temp = temp.isel(model_level=slice(-80, None))  #58 to 137

# # Extract the hour from time and filter
# log("Extracting valid_time hours...")
# hours = temp['valid_time'].dt.hour

# # Training: 00, 06, 12, 18
# log("Filtering training times (00, 06, 12, 18)...")
# train_hours = [0, 6, 12, 18]
# #train = temp.sel(valid_time=temp['valid_time'].where(hours.isin(train_hours), drop=True))

# # Testing: 01, 07, 13, 19
# log("Filtering testing times (01, 07, 13, 19)...")
# test_hours = [1, 7, 13, 19]
# test = temp.sel(valid_time=temp['valid_time'].where(hours.isin(test_hours), drop=True))

log("Done.")

[2025-04-17 10:14:05] Starting data loading...
[2025-04-17 10:14:05] Opening and combining NetCDF files...


/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/dask/array/core.py:4849: PerformanceWarning: Increasing number of chunks by factor of 137
  result = blockwise(


[2025-04-17 10:15:38] Extracting temperature variable...
[2025-04-17 10:15:38] Selecting model levels 58 to 137...
[2025-04-17 10:15:38] Done.


In [12]:
print("temp = \n", temp)

temp = 
 <xarray.DataArray 't' (valid_time: 32144, model_level: 80, latitude: 121,
                       longitude: 240)> Size: 299GB
dask.array<getitem, shape=(32144, 80, 121, 240), dtype=float32, chunksize=(22, 32, 31, 60), chunktype=numpy.ndarray>
Coordinates:
  * valid_time   (valid_time) datetime64[ns] 257kB 2014-01-01 ... 2024-12-31T...
  * model_level  (model_level) float64 640B 58.0 59.0 60.0 ... 135.0 136.0 137.0
  * latitude     (latitude) float64 968B 90.0 88.5 87.0 ... -87.0 -88.5 -90.0
  * longitude    (longitude) float64 2kB -180.0 -178.5 -177.0 ... 177.0 178.5
    expver       (valid_time) <U4 514kB dask.array<chunksize=(88,), meta=np.ndarray>
Attributes: (12/30)
    GRIB_paramId:                             130
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      29040
    GRIB_typeOfLevel:                         hybrid
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ..

In [4]:
import xarray as xr
import numpy as np
import xesmf as xe
import os
import hashlib

# Load ClimT grid
log("Loading ClimT grid...")
climt_load = np.load('/run/media/adamh/Elements/climt_lat_lon.npz')
climt_lat, climt_lon = climt_load['latitude'], climt_load['longitude']

# Create a new grid for the regridding
climt_grid = xr.Dataset(
    {
        'lat': (['lat'], climt_lat),
        'lon': (['lon'], climt_lon),
    }
)

def xr_regrid_all(ds, climt_grid, method='bilinear', weight_dir='/run/media/adamh/Elements/regrid_weights/'):
    """
    Regrid the entire dataset at once, without looping over individual time steps.
    """
    # Create a unique hash for the grid config to name the weight file
    grid_id = hashlib.md5((str(ds['latitude'].values.tobytes()) +
                           str(ds['longitude'].values.tobytes()) +
                           str(climt_grid['lat'].values.tobytes()) +
                           str(climt_grid['lon'].values.tobytes()) +
                           method).encode()).hexdigest()
    
    os.makedirs(weight_dir, exist_ok=True)
    weight_path = os.path.join(weight_dir, f'{method}_{grid_id}.nc')

    log(f"Creating regridder with weights file: {weight_path}")
    regridder = xe.Regridder(ds, climt_grid, method=method, periodic=True,
                             filename=weight_path,
                             reuse_weights=os.path.exists(weight_path))

    log("Regridding entire dataset...")
    regridded_ds = regridder(ds)

    # Convert pressure levels (if present)
    if 'pressure_level' in regridded_ds.coords:
        regridded_ds = regridded_ds.assign_coords(
            pressure_level=np.flip(regridded_ds.pressure_level.values) * 100
        )

    log("Regridding complete.")
    return regridded_ds


log("Regridding temperature data...")
temp_regridded = xr_regrid_all(temp, climt_grid)
log("Process complete.")

[2025-04-17 10:15:38] Loading ClimT grid...
[2025-04-17 10:15:39] Regridding temperature data...
[2025-04-17 10:15:39] Creating regridder with weights file: /run/media/adamh/Elements/regrid_weights/bilinear_0def5f10f091d2681bf0f92676d969c7.nc
[2025-04-17 10:15:39] Regridding entire dataset...


/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/xarray/core/computation.py:312: PerformanceWarning: Regridding is increasing the number of chunks by a factor of 9.0, you might want to specify sizes in `output_chunks` in the regridder call. Default behaviour is to preserve the chunk sizes from the input (31, 60).
  result_var = func(*data_vars)


[2025-04-17 10:15:47] Regridding complete.
[2025-04-17 10:15:47] Process complete.


In [10]:
hours = temp_regridded['valid_time'].dt.hour
input_hours = [0, 6, 12, 18]
input_regrid = temp_regridded.sel(valid_time=temp_regridded['valid_time'].where(hours.isin(input_hours), drop=True))

In [5]:
print(input_regrid)

NameError: name 'input_regrid' is not defined

In [14]:
# Ensure the DataArray has a name
if input_regrid.name is None:
    input_regrid.name = "t"

# Define the encoding dictionary for compression and chunk sizes
encoding = {
    "compression": "gzip",  # Compression type
    "compression_opts": 5,  # Compression level
    "chunksizes": (21, 32, 31, 60),  # Chunk sizes
}

# Apply encoding using the name of the DataArray
encoding_dict = {input_regrid.name: encoding}

# Write both datasets to disk
input_regrid.to_netcdf("/run/media/adamh/NV2/temp_w/ddc_ready_data/t_regrid_h5.nc", engine="h5netcdf", encoding=encoding_dict, compute=True)

In [1]:
import xarray as xr

aaa=xr.open_dataset("/run/media/adamh/NV2/temp_w/ddc_ready_data/t_regrid_h5.nc")
print(aaa)

/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


<xarray.Dataset> Size: 42GB
Dimensions:      (valid_time: 16072, model_level: 80, lat: 64, lon: 128)
Coordinates:
  * valid_time   (valid_time) datetime64[ns] 129kB 2014-01-01 ... 2024-12-31T...
  * model_level  (model_level) float64 640B 58.0 59.0 60.0 ... 135.0 136.0 137.0
    expver       (valid_time) <U4 257kB ...
  * lat          (lat) float64 512B 87.86 85.1 82.31 ... -82.31 -85.1 -87.86
  * lon          (lon) float64 1kB 0.0 2.812 5.625 8.438 ... 351.6 354.4 357.2
Data variables:
    t            (valid_time, model_level, lat, lon) float32 42GB ...


In [6]:
print(temp_regridded)

<xarray.DataArray (valid_time: 32144, model_level: 80, lat: 64, lon: 128)> Size: 84GB
dask.array<astype, shape=(32144, 80, 64, 128), dtype=float32, chunksize=(22, 32, 31, 60), chunktype=numpy.ndarray>
Coordinates:
  * valid_time   (valid_time) datetime64[ns] 257kB 2014-01-01 ... 2024-12-31T...
  * model_level  (model_level) float64 640B 58.0 59.0 60.0 ... 135.0 136.0 137.0
    expver       (valid_time) <U4 514kB dask.array<chunksize=(88,), meta=np.ndarray>
  * lat          (lat) float64 512B 87.86 85.1 82.31 ... -82.31 -85.1 -87.86
  * lon          (lon) float64 1kB 0.0 2.812 5.625 8.438 ... 351.6 354.4 357.2
Attributes:
    regrid_method:  bilinear


In [4]:
import zarr
import os
import numpy as np
from datetime import timedelta
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

output_dir = '/run/media/adamh/NV2/temp_w/training_pairs/'
os.makedirs(output_dir, exist_ok=True)

# Convert valid_time to pandas datetime for easier slicing
valid_times = pd.to_datetime(temp_regridded['valid_time'].values)

log("Starting Zarr saving over all daily pairs...")

# Function to process and save Zarr pairs
def save_zarr_pair(i, valid_times, temp_regridded):
    input_time = valid_times[i]
    target_time = valid_times[i + 1]
    
    # Only process if times match your desired hourly pattern
    if input_time.hour in [0, 6, 12, 18] and target_time - input_time == timedelta(hours=1):
        date_str = input_time.strftime('%Y-%m-%d')
        input_str = input_time.strftime('%H%M')
        target_str = target_time.strftime('%H%M')
        file_name = f"t_{date_str}_{input_str}_{target_str}.zarr"
        file_path = os.path.join(output_dir, file_name)
        
        # Skip if file already exists (check this before loading data)
        if os.path.exists(file_path):
            #log(f"Skipping {file_name}, file already exists.")
            return
        
        # If file does not exist, process and save
        input_data = temp_regridded.sel(valid_time=input_time).values
        target_data = temp_regridded.sel(valid_time=target_time).values
        
        # Add batch dimension
        input_data = np.expand_dims(input_data, axis=0)
        target_data = np.expand_dims(target_data, axis=0)
        
        # Save the data
        zarr_group = zarr.open(file_path, mode='w')
        zarr_group.create_dataset('input', data=input_data, chunks=(1, *input_data.shape[1:]), dtype='float32', compressor=zarr.Blosc(clevel=5))
        zarr_group.create_dataset('target', data=target_data, chunks=(1, *target_data.shape[1:]), dtype='float32', compressor=zarr.Blosc(clevel=5))
        
        log(f"Saved {file_name}")

# Function to run the parallelization
def parallel_save_zarr(valid_times, temp_regridded):
    with tqdm(total=len(valid_times) - 1, desc="Saving Zarr pairs") as pbar:
        with ThreadPoolExecutor(max_workers=8) as executor:  # Adjust max_workers based on your CPU capabilities
            futures = []
            for i in range(0, len(valid_times) - 1):
                futures.append(executor.submit(save_zarr_pair, i, valid_times, temp_regridded))
            
            # Update progress bar as the tasks complete
            for future in futures:
                future.result()  # Wait for each task to finish
                pbar.update(1)

# Call the parallel_save_zarr function to actually run the parallelized saving process
parallel_save_zarr(valid_times, temp_regridded)

log("Zarr saving completed for all valid daily pairs.")


[2025-04-17 03:54:45] Starting Zarr saving over all daily pairs...


Saving Zarr pairs:  58%|█████▊    | 18494/32143 [01:12<00:37, 365.28it/s]

[2025-04-17 03:55:58] Saved t_2014-02-19_1800_1900.zarr
[2025-04-17 03:56:00] Saved t_2024-02-16_1200_1300.zarr
[2025-04-17 03:56:03] Saved t_2024-02-17_1800_1900.zarr
[2025-04-17 03:56:06] Saved t_2024-02-16_1800_1900.zarr
[2025-04-17 03:56:06] Saved t_2024-02-17_0600_0700.zarr


Saving Zarr pairs:  92%|█████████▏| 29587/32143 [01:24<00:05, 506.48it/s]

[2025-04-17 03:56:10] Saved t_2024-02-16_0600_0700.zarr
[2025-04-17 03:56:12] Saved t_2024-02-17_1200_1300.zarr
[2025-04-17 03:56:15] Saved t_2024-02-17_0000_0100.zarr


...

[2025-04-17 07:01:07] Saved t_2024-12-31_0000_0100.zarr


Saving Zarr pairs: 100%|█████████▉| 32139/32143 [3:06:22<00:05,  1.41s/it]

[2025-04-17 07:01:08] Saved t_2024-12-31_0600_0700.zarr


Saving Zarr pairs: 100%|██████████| 32143/32143 [3:06:23<00:00,  2.87it/s]

[2025-04-17 07:01:08] Saved t_2024-12-31_1200_1300.zarr
[2025-04-17 07:01:08] Saved t_2024-12-31_1800_1900.zarr
[2025-04-17 07:01:08] Zarr saving completed for all valid daily pairs.


In [5]:
import numpy as np
import xarray as xr
import dask
from dask.array import from_array
import time

# Assuming you already have your temp_regridded DataArray
# e.g., temp_regridded = xr.open_dataset(..., chunks=...)

data = temp_regridded  # your original big array
n = data.sizes['valid_time']
chunk_size = 500

# Initialise accumulators
sum_ = 0
sum_sq = 0
count = 0

start_time = time.time()
print("Computing global mean and std with manual chunking...")

for i in range(0, n, chunk_size):
    chunk = data.isel(valid_time=slice(i, min(i + chunk_size, n))).load()
    np_chunk = chunk.values.astype(np.float64)
    sum_ += np_chunk.sum()
    sum_sq += (np_chunk ** 2).sum()
    count += np_chunk.size

    print(f"Processed {min(i + chunk_size, n)} / {n} timesteps")

# Final stats
global_mean = sum_ / count
global_std = np.sqrt((sum_sq / count) - (global_mean ** 2))

elapsed = time.time() - start_time
print(f"\n✅ Done in {elapsed:.2f}s")
print(f"Global mean: {global_mean:.5f}")
print(f"Global std: {global_std:.5f}")


Computing global mean and std with manual chunking...
Processed 500 / 32144 timesteps
Processed 1000 / 32144 timesteps
Processed 1500 / 32144 timesteps
Processed 2000 / 32144 timesteps
Processed 2500 / 32144 timesteps
Processed 3000 / 32144 timesteps
Processed 3500 / 32144 timesteps
Processed 4000 / 32144 timesteps
Processed 4500 / 32144 timesteps
Processed 5000 / 32144 timesteps
Processed 5500 / 32144 timesteps
Processed 6000 / 32144 timesteps
Processed 6500 / 32144 timesteps
Processed 7000 / 32144 timesteps
Processed 7500 / 32144 timesteps
Processed 8000 / 32144 timesteps
Processed 8500 / 32144 timesteps
Processed 9000 / 32144 timesteps
Processed 9500 / 32144 timesteps
Processed 10000 / 32144 timesteps
Processed 10500 / 32144 timesteps
Processed 11000 / 32144 timesteps
Processed 11500 / 32144 timesteps
Processed 12000 / 32144 timesteps
Processed 12500 / 32144 timesteps
Processed 13000 / 32144 timesteps
Processed 13500 / 32144 timesteps
Processed 14000 / 32144 timesteps
Processed 1450

In [2]:
import torch
import xarray as xr
from tqdm import tqdm

def convert_and_save_yearly_tensors(nc_path, varname, prefix, save_dir):
    """
    Load NetCDF, split data by year, convert to PyTorch tensors, and save each year separately.
    """
    ds = xr.open_dataset(nc_path, engine="h5netcdf")
    dataarray = ds[varname]
    years = sorted(set(dataarray["valid_time.year"].values))

    for year in tqdm(years, desc=f"Processing {prefix}"):
        yearly_data = dataarray.sel(valid_time=str(year))
        tensor = torch.tensor(yearly_data.values).unsqueeze(1)  # shape: (time, 1, level, lat, lon)
        save_path = f"{save_dir}/{prefix}_{year}.pt"
        torch.save(tensor, save_path)
        print(f"Saved: {save_path} | Shape: {tensor.shape}")

    ds.close()  # explicitly close the dataset

# ----------------- Main -----------------

base_dir = "/run/media/adamh/Elements/toy_nn"
save_dir = "/run/media/adamh/NV2/temp_w"

# Paths to files and variable name
train_path = f"{base_dir}/t_train_regrid_h5.nc"
test_path = f"{base_dir}/t_test_regrid_h5.nc"
varname = "variable_name"  # Replace this with the actual variable name if needed

# Process both datasets
convert_and_save_yearly_tensors(train_path, varname, "t_train_tensor", save_dir)
convert_and_save_yearly_tensors(test_path, varname, "t_test_tensor", save_dir)


Processing t_train_tensor:   9%|▉         | 1/11 [01:47<17:56, 107.62s/it]

Saved: /run/media/adamh/NV2/temp_w/t_train_tensor_2014.pt | Shape: torch.Size([1460, 1, 77, 64, 128])


Processing t_train_tensor:  18%|█▊        | 2/11 [03:34<16:05, 107.26s/it]

Saved: /run/media/adamh/NV2/temp_w/t_train_tensor_2015.pt | Shape: torch.Size([1460, 1, 77, 64, 128])


Processing t_train_tensor:  27%|██▋       | 3/11 [05:19<14:10, 106.31s/it]

...

Saved: /run/media/adamh/NV2/temp_w/t_test_tensor_2022.pt | Shape: torch.Size([1460, 1, 77, 64, 128])


Processing t_test_tensor:  91%|█████████ | 10/11 [17:42<01:45, 105.96s/it]

Saved: /run/media/adamh/NV2/temp_w/t_test_tensor_2023.pt | Shape: torch.Size([1460, 1, 77, 64, 128])


Processing t_test_tensor: 100%|██████████| 11/11 [19:28<00:00, 106.22s/it]

Saved: /run/media/adamh/NV2/temp_w/t_test_tensor_2024.pt | Shape: torch.Size([1464, 1, 77, 64, 128])


In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import zarr
import os
import numpy as np

class ZarrForecastDataset(Dataset):
    def __init__(self, root_dir, mean, std):
        self.root_dir = root_dir
        self.file_names = sorted([f for f in os.listdir(root_dir) if f.endswith('.zarr')])
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        zarr_path = os.path.join(self.root_dir, self.file_names[idx])
        input_array = zarr.open(os.path.join(zarr_path, "input"), mode='r')[0]
        target_array = zarr.open(os.path.join(zarr_path, "target"), mode='r')[0]

        input_tensor = torch.from_numpy(input_array).float().unsqueeze(0)
        target_tensor = torch.from_numpy(target_array).float().unsqueeze(0)

        # Normalize
        input_tensor = (input_tensor - self.mean) / self.std
        target_tensor = (target_tensor - self.mean) / self.std

        return input_tensor, target_tensor

# Assuming global_mean and global_std are already defined as floats or np.ndarrays
global_mean, global_std = 249.10009, 29.99640
dataset = ZarrForecastDataset("/run/media/adamh/NV2/temp_w/training_pairs/", global_mean, global_std)

dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)

# Sanity check
# for i, (x, y) in enumerate(dataloader):
#     print(f"Batch {i}: input shape = {x.shape}, target shape = {y.shape}")
#     if i == 0:
#         break


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Weather_UNet3D(nn.Module):
    def __init__(self, dropout_rate=0.2, base_param=16, geo_rat=2, device=None):
        super(Weather_UNet3D, self).__init__()
        self.device = device if device is not None else torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.dropout_rate = dropout_rate
        self.base_param = base_param
        self.geo_rat = geo_rat

        # Encoder Block 1
        lvl1_param = int(base_param)
        self.conv1_1 = nn.Conv3d(1, lvl1_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn1_1 = nn.BatchNorm3d(lvl1_param)
        self.relu1_1 = nn.ReLU()
        self.dropout1_1 = nn.Dropout3d(p=dropout_rate)
        
        self.conv1_2 = nn.Conv3d(lvl1_param, lvl1_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn1_2 = nn.BatchNorm3d(lvl1_param)
        self.relu1_2 = nn.ReLU()
        self.dropout1_2 = nn.Dropout3d(p=dropout_rate)
        
        self.pool1 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))  # Pooling only on spatial dims

        # Encoder Block 2
        lvl2_param = int(geo_rat * lvl1_param)
        self.conv2_1 = nn.Conv3d(lvl1_param, lvl2_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn2_1 = nn.BatchNorm3d(lvl2_param)
        self.relu2_1 = nn.ReLU()
        self.dropout2_1 = nn.Dropout3d(p=dropout_rate)
        
        self.conv2_2 = nn.Conv3d(lvl2_param, lvl2_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn2_2 = nn.BatchNorm3d(lvl2_param)
        self.relu2_2 = nn.ReLU()
        self.dropout2_2 = nn.Dropout3d(p=dropout_rate)
        
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))  # Pooling only on spatial dims

        # Encoder Block 3
        lvl3_param = int(geo_rat * lvl2_param)
        self.conv3_1 = nn.Conv3d(lvl2_param, lvl3_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn3_1 = nn.BatchNorm3d(lvl3_param)
        self.relu3_1 = nn.ReLU()
        self.dropout3_1 = nn.Dropout3d(p=dropout_rate)
        
        self.conv3_2 = nn.Conv3d(lvl3_param, lvl3_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn3_2 = nn.BatchNorm3d(lvl3_param)
        self.relu3_2 = nn.ReLU()
        self.dropout3_2 = nn.Dropout3d(p=dropout_rate)
        
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))  # Pooling only on spatial dims

        # Bottleneck
        lvl4_param = int(geo_rat * lvl3_param)
        self.conv4_1 = nn.Conv3d(lvl3_param, lvl4_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn4_1 = nn.BatchNorm3d(lvl4_param)
        self.relu4_1 = nn.ReLU()
        self.dropout4_1 = nn.Dropout3d(p=dropout_rate)
        
        self.conv4_2 = nn.Conv3d(lvl4_param, lvl4_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn4_2 = nn.BatchNorm3d(lvl4_param)
        self.relu4_2 = nn.ReLU()
        self.dropout4_2 = nn.Dropout3d(p=dropout_rate)

        # Decoder Block 1 (Upsampling)
        self.upconv1 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False)
        self.conv5_1 = nn.Conv3d(lvl4_param + lvl3_param, lvl3_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn5_1 = nn.BatchNorm3d(lvl3_param)
        self.relu5_1 = nn.ReLU()
        self.dropout5_1 = nn.Dropout3d(p=0.3)

        self.conv5_2 = nn.Conv3d(lvl3_param, lvl3_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn5_2 = nn.BatchNorm3d(lvl3_param)
        self.relu5_2 = nn.ReLU()
        self.dropout5_2 = nn.Dropout3d(p=dropout_rate)

        # Decoder Block 2 (Upsampling)
        self.upconv2 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False)
        self.conv6_1 = nn.Conv3d(lvl3_param + lvl2_param, lvl2_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn6_1 = nn.BatchNorm3d(lvl2_param)
        self.relu6_1 = nn.ReLU()
        self.dropout6_1 = nn.Dropout3d(p=dropout_rate)

        self.conv6_2 = nn.Conv3d(lvl2_param, lvl2_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn6_2 = nn.BatchNorm3d(lvl2_param)
        self.relu6_2 = nn.ReLU()
        self.dropout6_2 = nn.Dropout3d(p=dropout_rate)

        # Decoder Block 3 (Upsampling)
        self.upconv3 = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=False)
        self.conv7_1 = nn.Conv3d(lvl2_param + lvl1_param, lvl1_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn7_1 = nn.BatchNorm3d(lvl1_param)
        self.relu7_1 = nn.ReLU()
        self.dropout7_1 = nn.Dropout3d(p=dropout_rate)

        self.conv7_2 = nn.Conv3d(lvl1_param, lvl1_param, kernel_size=3, padding=1, padding_mode='circular')
        self.bn7_2 = nn.BatchNorm3d(lvl1_param)
        self.relu7_2 = nn.ReLU()
        self.dropout7_2 = nn.Dropout3d(p=dropout_rate)

        # Final output layer
        self.conv8 = nn.Conv3d(lvl1_param, 1, kernel_size=3, padding=1, padding_mode='circular')

    def forward(self, x):
        x0 = x.to(self.device)
        
        # Encoder
        x1 = self.conv1_1(x0)
        x1 = self.bn1_1(x1)
        x1 = self.relu1_1(x1)
        x1 = self.dropout1_1(x1)
        
        x1 = self.conv1_2(x1)
        x1 = self.bn1_2(x1)
        x1 = self.relu1_2(x1)
        x1 = self.dropout1_2(x1)
        
        p1 = self.pool1(x1)
        
        x2 = self.conv2_1(p1)
        x2 = self.bn2_1(x2)
        x2 = self.relu2_1(x2)
        x2 = self.dropout2_1(x2)
        
        x2 = self.conv2_2(x2)
        x2 = self.bn2_2(x2)
        x2 = self.relu2_2(x2)
        x2 = self.dropout2_2(x2)
        
        p2 = self.pool2(x2)
        
        x3 = self.conv3_1(p2)
        x3 = self.bn3_1(x3)
        x3 = self.relu3_1(x3)
        x3 = self.dropout3_1(x3)
        
        x3 = self.conv3_2(x3)
        x3 = self.bn3_2(x3)
        x3 = self.relu3_2(x3)
        x3 = self.dropout3_2(x3)
        
        p3 = self.pool3(x3)
        
        b = self.conv4_1(p3)
        b = self.bn4_1(b)
        b = self.relu4_1(b)
        b = self.dropout4_1(b)
        
        b = self.conv4_2(b)
        b = self.bn4_2(b)
        b = self.relu4_2(b)
        b = self.dropout4_2(b)

        # Decoder
        u1 = self.upconv1(b)
        u1 = torch.cat([u1, x3], dim=1)  # Skip connection from Encoder Block 3
        u1 = self.conv5_1(u1)
        u1 = self.bn5_1(u1)
        u1 = self.relu5_1(u1)
        u1 = self.dropout5_1(u1)
        
        u1 = self.conv5_2(u1)
        u1 = self.bn5_2(u1)
        u1 = self.relu5_2(u1)
        u1 = self.dropout5_2(u1)
        
        u2 = self.upconv2(u1)
        u2 = torch.cat([u2, x2], dim=1)  # Skip connection from Encoder Block 2
        u2 = self.conv6_1(u2)
        u2 = self.bn6_1(u2)
        u2 = self.relu6_1(u2)
        u2 = self.dropout6_1(u2)
        
        u2 = self.conv6_2(u2)
        u2 = self.bn6_2(u2)
        u2 = self.relu6_2(u2)
        u2 = self.dropout6_2(u2)
        
        u3 = self.upconv3(u2)
        u3 = torch.cat([u3, x1], dim=1)  # Skip connection from Encoder Block 1
        u3 = self.conv7_1(u3)
        u3 = self.bn7_1(u3)
        u3 = self.relu7_1(u3)
        u3 = self.dropout7_1(u3)
        
        u3 = self.conv7_2(u3)
        u3 = self.bn7_2(u3)
        u3 = self.relu7_2(u3)
        u3 = self.dropout7_2(u3)

        out = self.conv8(u3)
        
        return out

In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model
model = Weather_UNet3D(dropout_rate=0.2, base_param=32, geo_rat=2, device=device).to(device)
model.eval()

# Get one real batch from your DataLoader
for x, y in dataloader:
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        out = model(x)
    print("Input shape: ", x.shape)
    print("Target shape:", y.shape)
    print("Output shape:", out.shape)
    break  # just test with the first batch


Input shape:  torch.Size([4, 1, 80, 64, 128])
Target shape: torch.Size([4, 1, 80, 64, 128])
Output shape: torch.Size([4, 1, 80, 64, 128])


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import optuna
import os
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast
import joblib  # Import joblib for saving and loading the study

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === DEBUG MODE ===
debug_mode = True
save_dir = "/run/media/adamh/NV2/temp_w/training_results"
os.makedirs(save_dir, exist_ok=True)

# Dummy validate function (replace with your actual logic)
def validate(model, val_loader, criterion):
    model.eval()
    val_loss, val_rmse, val_mae = 0.0, 0.0, 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, targets)
            val_loss += loss.item()
            val_rmse += torch.sqrt(nn.functional.mse_loss(outputs, targets)).item()
            val_mae += nn.functional.l1_loss(outputs, targets).item()

    n = len(val_loader)
    return val_loss / n, val_rmse / n, val_mae / n

class EarlyStopping:
    def __init__(self, patience=10, min_delta=0.0001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = np.inf
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        return self.early_stop

# Define the objective function for Optuna
def objective(trial):
    # === Hyperparameters ===
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    base_param = trial.suggest_int('base_param', 16, 64, step=4)
    geo_rat = trial.suggest_float('geo_rat', 1.0, 3.0, step=0.25)
    learning_rate = trial.suggest_float('learning_rate', 1e-6, 1e-2, log=True)
    batch_size = trial.suggest_int('batch_size', 4, 64, step=4)
    num_epochs = 3 if debug_mode else 10000

    model = Weather_UNet3D(dropout_rate=dropout_rate, base_param=base_param, geo_rat=geo_rat, device=device).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

    dataset = ZarrForecastDataset("/run/media/adamh/NV2/temp_w/training_pairs/", global_mean, global_std)
    if debug_mode:
        dataset = torch.utils.data.Subset(dataset, range(10))

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    early_stopping = EarlyStopping(patience=3 if debug_mode else 10, min_delta=0.0001)
    scaler = GradScaler(enabled=torch.cuda.is_available())

    tb_dir = os.path.join(save_dir, "tensorboard")
    os.makedirs(tb_dir, exist_ok=True)
    writer = SummaryWriter(log_dir=tb_dir)

    best_val_loss = float('inf')
    train_losses, val_losses = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for i, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, targets)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            torch.cuda.empty_cache()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        writer.add_scalar("Loss/train", avg_train_loss, epoch)
        print(f"Epoch [{epoch+1}/{num_epochs}] Training Loss: {avg_train_loss:.6f}")

        avg_val_loss, avg_val_rmse, avg_val_mae = validate(model, val_loader, criterion)
        val_losses.append(avg_val_loss)
        writer.add_scalar("Loss/val", avg_val_loss, epoch)
        writer.add_scalar("Val/RMSE", avg_val_rmse, epoch)
        writer.add_scalar("Val/MAE", avg_val_mae, epoch)

        scheduler.step(avg_val_loss)

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            model_save_path = os.path.join(save_dir, 'debug_model.pth' if debug_mode else 'best_model.pth')
            torch.save(model.state_dict(), model_save_path)

        if early_stopping(avg_val_loss):
            print("Early stopping triggered!")
            break

    writer.close()
    np.save(os.path.join(save_dir, "train_losses.npy"), np.array(train_losses))
    np.save(os.path.join(save_dir, "val_losses.npy"), np.array(val_losses))

    return best_val_loss

# Global stats
global_mean, global_std = 249.10009, 29.99640

# Optuna
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=1 if debug_mode else 100)

print(f"Best trial: {study.best_trial}")
print(f"Best hyperparameters: {study.best_trial.params}")

# Save study using joblib
study_path = os.path.join(save_dir, 'weather_unet3d_hyperopt_test.joblib' if debug_mode else 'weather_unet3d_hyperopt.joblib')
joblib.dump(study, study_path)

print(f"Study saved at {study_path}")

[I 2025-04-19 09:27:44,152] A new study created in memory with name: no-name-44d309b0-c294-4f30-b62c-271351ad0320
/tmp/ipykernel_7492/1296081919.py:81: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler(enabled=torch.cuda.is_available())
/tmp/ipykernel_7492/1296081919.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/home/adamh/miniconda3/envs/x_env/lib/python3.10/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


Epoch [1/3] Training Loss: 1.027122


/tmp/ipykernel_7492/1296081919.py:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [2/3] Training Loss: 1.074489
Epoch [3/3] Training Loss: 1.046483


[I 2025-04-19 09:30:20,230] Trial 0 finished with value: 0.9342626333236694 and parameters: {'dropout_rate': 0.35744594383141237, 'base_param': 60, 'geo_rat': 1.0, 'learning_rate': 6.897350314371482e-06, 'batch_size': 64}. Best is trial 0 with value: 0.9342626333236694.


Best trial: FrozenTrial(number=0, state=TrialState.COMPLETE, values=[0.9342626333236694], datetime_start=datetime.datetime(2025, 4, 19, 9, 27, 44, 152804), datetime_complete=datetime.datetime(2025, 4, 19, 9, 30, 20, 230496), params={'dropout_rate': 0.35744594383141237, 'base_param': 60, 'geo_rat': 1.0, 'learning_rate': 6.897350314371482e-06, 'batch_size': 64}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'dropout_rate': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'base_param': IntDistribution(high=64, log=False, low=16, step=4), 'geo_rat': FloatDistribution(high=3.0, log=False, low=1.0, step=0.25), 'learning_rate': FloatDistribution(high=0.01, log=True, low=1e-06, step=None), 'batch_size': IntDistribution(high=64, log=False, low=4, step=4)}, trial_id=0, value=None)
Best hyperparameters: {'dropout_rate': 0.35744594383141237, 'base_param': 60, 'geo_rat': 1.0, 'learning_rate': 6.897350314371482e-06, 'batch_size': 64}
Study saved at /run/media/adam